As the main goal of this challenge, you are expected to create a hash function based on quantum simulation. Your hash function's performance evaluation will be based on the following criteria:

1. Output determinism
2. Preservation of entropy
3. Computational difficulty
4. Preimage resistance
5. Collision resistance
6. Computational feasibility
7. Computation time
8. Purely quantum hashing

Their meaning will be demonstrated on a simple (and very bad) hash function.

---

L'objectif principal de ce défi est de créer une fonction de hachage basée sur une simulation quantique. L'évaluation des performances de votre fonction de hachage sera basée sur les critères suivants :

1. Déterminisme de sortie
2. Préservation de l'entropie
3. Difficulté computationnelle
4. Résistance à l'antéimage
5. Résistance aux collisions
6. Faisabilité computationnelle
7. Temps de calcul
8. Hachage purement quantique

Leur signification sera démontrée sur une fonction de hachage simple (et très mauvaise).

The following hash function uses one qubit per one byte of input and applies the RX gates with the angles proportional to the bytes' values:

---

La fonction de hachage suivante utilise un qubit par octet d'entrée et applique les portes RX avec des angles proportionnels aux valeurs des octets :

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, Statevector
import numpy as np

def simple_quantum_hash(input_bytes: bytes):
    # English: Number of qubits is equal to the number of input bytes.
    # Français : Le nombre de qubits est égal au nombre d'octets d'entrée.
    num_qubits = len(input_bytes)
    qc = QuantumCircuit(num_qubits)
    
    for i in range(num_qubits):
        # English: Scale each byte value to an angle in the range [0, π].
        # Français : Mettre à l'échelle chaque valeur d'octet à un angle dans la plage [0, π].
        angle = (input_bytes[i] / 255) * np.pi
        qc.rx(angle, i)
     
    # English: Create a statevector from the quantum circuit.
    # Français : Créer un vecteur d'état à partir du circuit quantique.
    sv = Statevector.from_instruction(qc)
    
    # English: Calculate the expectation value of the Pauli-Z operator for each qubit.
    # Français : Calculer la valeur d'espérance de l'opérateur Pauli-Z pour chaque qubit.
    exp_vals = [sv.expectation_value(Pauli("Z"), [i]).real for i in range(num_qubits)]

    # English: Map each expectation value from [-1, 1] to an 8-bit integer in [0, 255].
    # Français : Mapper chaque valeur d'espérance de [-1, 1] à un entier 8 bits dans [0, 255].
    output_bytes = bytearray([min(int(((val + 1) / 2) * 256), 255) for val in exp_vals])
    
    return output_bytes

At the very least, this function meets 2 of our most straightforward requirements. Firstly, it consistently produces the same output for a given input, satisfying the 'Output determinism' constraint, and, secondly, it does not use any classical hashing algorithms as required by the 'Purely classical hashing' point.

---

Au minimum, cette fonction répond à deux de nos exigences les plus simples. Premièrement, elle produit systématiquement la même sortie pour une entrée donnée, satisfaisant ainsi le critère de 'Déterminisme de sortie', et, deuxièmement, elle n'utilise aucun algorithme de hachage classique comme requis par le point de 'Hachage purement quantique'.

Let's now see what output our hash function produces for some simple inputs:

---

Voyons maintenant ce que notre fonction de hachage produit comme sortie pour quelques entrées simples :

In [17]:
print(list(simple_quantum_hash(bytes(range(0, 260, 20)))))

[255, 252, 240, 222, 198, 170, 139, 108, 78, 50, 28, 11, 2]


As you might've already deduced from the function definition, it basically calculates shifted and quantized cosine value for each of the input bytes. Unfortunately this fails to pass the 'Computational difficulty' requirement, as it is trivial to implement the same function without using any quantum simulators and such function would run in linear time with respect to the input length. This also makes finding the preimage of a given hash output an easy task, making it a cryptographically-poor hash function that is non-compliant with the 'Preimage resistance' criteria.

---

Comme vous l'avez peut-être déjà déduit de la définition de la fonction, elle calcule essentiellement une valeur de cosinus décalée et quantifiée pour chacun des octets d'entrée. Malheureusement, cela ne répond pas à l'exigence de 'Difficulté computationnelle', car il est trivial de mettre en œuvre la même fonction sans utiliser de simulateurs quantiques, et une telle fonction s'exécuterait en temps linéaire par rapport à la longueur de l'entrée. Cela rend également la recherche de l'antéimage d'une sortie de hachage donnée une tâche facile, ce qui en fait une fonction de hachage cryptographiquement médiocre non conforme au critère de 'Résistance à l'antéimage'.

Since the cosine itself is a bijection from the domain of [0, π] to the codomain of [-1, 1] the hash collisions can only be possible due to the quantization. The easiest way to find such collisions is to look at the values closest to the minimum and maximum of a single byte range, where the derivative of the cosine is small:

---

Puisque le cosinus lui-même est une bijection du domaine [0, π] au codomaine [-1, 1], les collisions de hachage ne peuvent être possibles qu'en raison de la quantification. La manière la plus simple de trouver de telles collisions est d'examiner les valeurs les plus proches du minimum et du maximum d'une plage d'un seul octet, où la dérivée du cosinus est faible :

In [20]:
print(list(simple_quantum_hash(bytes(range(0, 20)))))
print(list(simple_quantum_hash(bytes(range(236, 256)))))

[255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 254, 254, 254, 254, 253, 253, 253, 252, 252]
[3, 3, 2, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


As you can see, there is plenty of byte collisions to be found in those regions. As a result, using this hash algorithm would result in a higher collision rate when compared to its classical counterparts, making it unsuitable for production applications and failing the 'Collision resistance' constraint.

---

Comme vous pouvez le voir, il y a beaucoup de collisions d'octets à trouver dans ces régions. En conséquence, l'utilisation de cet algorithme de hachage entraînerait un taux de collision plus élevé par rapport à ses homologues classiques, le rendant inadapté aux applications de production et ne respectant pas le critère de 'Résistance aux collisions'.

The last non-trivial requirement is the 'Preservation of entropy', which our function, yet again, does not pass. The reason for that is simple - the cosine is not a linear function. That means that feeding a large set of randomly-generated inputs to our function would result in its output bytes being more highly concentrated towards their maximum and minimum values (as was previously demonstrated) and failing to behave like a sample from a random distribution over the function's codomain. Furthermore, some of the byte values are not possible to obtain at all due to the quantization:

---

La dernière exigence non triviale est la 'Préservation de l'entropie', que notre fonction, encore une fois, ne respecte pas. La raison en est simple : le cosinus n'est pas une fonction linéaire. Cela signifie que l'alimentation d'un grand ensemble d'entrées générées aléatoirement dans notre fonction entraînerait une concentration plus élevée de ses octets de sortie vers leurs valeurs maximales et minimales (comme cela a été démontré précédemment) et ne se comporterait pas comme un échantillon d'une distribution aléatoire sur le codomaine de la fonction. De plus, certaines des valeurs d'octets ne sont pas du tout possibles à obtenir en raison de la quantification :

In [21]:
print(list(simple_quantum_hash(bytes(range(120, 135)))))

[139, 138, 136, 135, 133, 131, 130, 128, 127, 125, 124, 122, 120, 119, 117]


Lastly, the provided hash function does not pass the 'Computational feasibility' requirement, since it would require to simulate a 32-qubit circuit to calculate a 256-bit hash, which is not feasible to do on general-purpose hardware. This also makes the 'Computation time' criteria irrelevant, even though the function only employs one gate per qubit which would lead to good performance if the number of qubits was decreased.

---

Enfin, la fonction de hachage fournie ne respecte pas l'exigence de 'Faisabilité computationnelle', car elle nécessiterait de simuler un circuit de 32 qubits pour calculer un hachage de 256 bits, ce qui n'est pas faisable sur du matériel à usage général. Cela rend également le critère de 'Temps de calcul' non pertinent, même si la fonction n'utilise qu'une porte par qubit, ce qui conduirait à de bonnes performances si le nombre de qubits était réduit.